### The Fermi-Hubbard Model

This notebook shows how to use the `tensor_basis` constructor to build the Hamiltonian of interacting spinful fermions in 1d, desctibed by the Fermi-Hubbard model (FHM):
$$H = -J\sum_{i=0,\sigma}^{L-1} \left(c^\dagger_{i\sigma}c_{i+1,\sigma} - c_{i\sigma}c^\dagger_{i+1,\sigma}\right) - \mu\sum_{i=0,\sigma}^{L-1} n_{i\sigma}  +U\sum_{i=0}^{L-1} n_{i\uparrow }n_{i\downarrow } $$
where $J$ is the hopping matrix element, $\mu$: the chemical potential, and $U$ -- the onsite $s$-wave interaction. 

We begin by loading the libraries and defining the model parameters:

In [1]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import spinless_fermion_basis_1d, tensor_basis # Hilbert space fermion and tensor bases
import numpy as np # generic math functions
##### define model parameters #####
L=4 # system size
J=1.0 # hopping
U=np.sqrt(2.0) # interaction
mu=0.0 # chemical potential

To build the basis for spinful fermions, we take two copies of the basis for spinless fermions and tensor them using the `tensor_basis` constructor. While the `tensor_basis` can be used to tensor any two bases objects, it does not allow for passing symmetries, other than particle number conservation (we are currently working on developing a separate class which will allow using all symmetries for spinful fermions). 

To this end, we define the number of spin-up and spin-down fermions, and proceed as follows:

In [3]:
# define boson basis with 3 states per site L bosons in the lattice
N_up = L//2 + L % 2 # number of fermions with spin up
N_down = L//2 # number of fermions with spin down
basis_up=spinless_fermion_basis_1d(L,Nf=N_up)
basis_down=spinless_fermion_basis_1d(L,Nf=N_down)
basis = tensor_basis(basis_up,basis_down) # spinful fermions
print(basis_up)
print('hello world')
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
     0.         |1 1 0 0>           12  
     1.         |1 0 1 0>           10  
     2.         |1 0 0 1>            9  
     3.         |0 1 1 0>            6  
     4.         |0 1 0 1>            5  
     5.         |0 0 1 1>            3  
hello world
reference states: 
array index   /   Fock state   /   integer repr. 
	 0.         |1 1 0 0>           12         |1 1 0 0>           12  
	 1.         |1 1 0 0>           12         |1 0 1 0>           10  
	 2.         |1 1 0 0>           12         |1 0 0 1>            9  
	 3.         |1 1 0 0>           12         |0 1 1 0>            6  
	 4.         |1 1 0 0>           12         |0 1 0 1>            5  
	 5.         |1 1 0 0>           12         |0 0 1 1>            3  
	 6.         |1 0 1 0>           10         |1 1 0 0>           12  
	 7.         |1 0 1 0>           10         |1 0 1 0>           10  
	 8.         |1 0 1 0>           10         |1 0 0 1

Alternatively, one can use the `spinful_fermion_basis_1d` class as well. This class, unlike the `tensor_basis` class can handle various 1d symmetries in the usual way and should be preferred for dealing with the FHM.

In [4]:
from quspin.basis import spinful_fermion_basis_1d

basis = spinful_fermion_basis_1d(L,Nf=(N_up,N_down))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |1 1 0 0>|1 1 0 0>           204  
      1.         |1 1 0 0>|1 0 1 0>           202  
      2.         |1 1 0 0>|1 0 0 1>           201  
      3.         |1 1 0 0>|0 1 1 0>           198  
      4.         |1 1 0 0>|0 1 0 1>           197  
      5.         |1 1 0 0>|0 0 1 1>           195  
      6.         |1 0 1 0>|1 1 0 0>           172  
      7.         |1 0 1 0>|1 0 1 0>           170  
      8.         |1 0 1 0>|1 0 0 1>           169  
      9.         |1 0 1 0>|0 1 1 0>           166  
     10.         |1 0 1 0>|0 1 0 1>           165  
     11.         |1 0 1 0>|0 0 1 1>           163  
     12.         |1 0 0 1>|1 1 0 0>           156  
     13.         |1 0 0 1>|1 0 1 0>           154  
     14.         |1 0 0 1>|1 0 0 1>           153  
     15.         |1 0 0 1>|0 1 1 0>           150  
     16.         |1 0 0 1>|0 1 0 1>           149  
     17.         |1 0 0 1>|0 0 1 1>           1

Defining the site-coupling lists is the same as before (mind the signs in the fermion hopping operator, though!). 

The `tensor_basis` accepts extended operator strings. The idea is that within the subspace of each basis, we use the operator strings belonging to the corresponding underlying basis (for `spinless_fermion_basis_1d`, the allowed operators are `"+"`, `"-"`, `"n"`, and `"I"`). We then use a `...|...` to separate the operators that act on spin-up (left) and spin-down (right). 

For instance, the hopping operators $c_{j,\uparrow}c^\dagger_{j+1,\uparrow}$ and $c_{j,\downarrow}c^\dagger_{j+1,\downarrow}$ are represented as `'-+|I'` and `'I|-+'`, repsectively, where `'I'` stands for the identity (and can be dropped, see below). On the other hand, the spin-flip hopping process $c_{j,\uparrow}c^\dagger_{j+1,\downarrow}$ would mix the spin-up and spin-down sectors and would take the form `'-|+'`.

In [11]:
# define site-coupling lists
hop_right=[[-J,i,(i+1)%L] for i in range(L)] #PBC
hop_left= [[+J,i,(i+1)%L] for i in range(L)] #PBC 
pot=[[-mu,i] for i in range(L)] # -\mu \sum_j n_{j \sigma}
interact=[[U,i,i] for i in range(L)] # U/2 \sum_j n_{j,up} n_{j,down}
# define static and dynamic lists
static=[
        ['+-|',hop_left],  # up hops left
        ['-+|',hop_right], # up hops right
        ['|+-',hop_left],  # down hops left
        ['|-+',hop_right], # down hops right
        ['n|',pot],        # up on-site potention
        ['|n',pot],        # down on-site potention
        ['n|n',interact]   # up-down interaction
                                ]
dynamic=[]
# build Hamiltonian
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
H=hamiltonian(static,dynamic,basis=basis,dtype=np.float64,**no_checks)

In [18]:
# define the impurity Anderson model
# define sit-coupling lists
mu_i = 0.0
mu_f = 0.0
hop_up = [[-J, 0, i] for i in range(1, L)]
hop_down = [[J, i, 0] for i in range(1, L)]
pot_i = [[mu_i, i] for i in range(1, L)]
pot_f =[[-mu_f, 0]]
h_interact = [[U, 0, 0]]    # the local Hubbard interaction
# # define static and dynamic lists
a_static=[
    ['+-|', hop_up],    # up hops up
    ['-+|', hop_down],  # up hops down
    ['|+-', hop_up],    # down hops up
    ['|-+', hop_down],  # down hops down
    ['n|', pot_i],      # up on-bath-site potention
    ['|n', pot_i],      # down on-bath-site potention
    ['n|', pot_f],      # up impurity-site potention
    ['|n', pot_f],      # down impurity-site potention
    ['n|n', h_interact]
]
dynamic=[]
# build Hamiltonian
no_checks = dict(check_pcon=False, check_symm=True, check_herm=True)
H_Anderson=hamiltonian(static, dynamic, basis=basis, dtype=np.float64, **no_checks)

Hermiticity check passed!


C:\ProgramData\Anaconda3\envs\quspin\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.basis_1d.fermion.spinful_fermion_basis_1d'>, to turn off this warning set check_symm=False in hamiltonian


In [20]:
##### various exact diagonalisation routines #####
# calculate entire spectrum only
E = H_Anderson.eigvalsh()
# calculate full eigensystem
E, V = H_Anderson.eigh()
# calculate minimum and maximum energy only
Emin, Emax = H_Anderson.eigsh(k=2, which="BE", maxiter=1E4, return_eigenvectors=False)
# calculate the eigenstate closest to energy E_star
E_star = 0.0
# E, psi_0 = H_Anderson.eigsh(k=1, sigma=E_star, maxiter=1E4)
# psi_0 = psi_0.reshape((-1,))
print('end')

end
